In [2]:
p = 2**255 - 19
d = 0x52036cee2b6ffe738cc740797779e89800700a4d4141d8ab75eb4dca135978a3

In [ ]:
import hashlib
def sha512(m):
    return int(hashlib.sha512(m).hexdigest(),16)

In [27]:
import random as rn
print(hex(rn.randint(0, 2**256-1)))

0x614e0689ead0557c59c2d527a39abc74ffcc5ebf57d384276e9f3e65e2e43dae


In [ ]:
R = rn.randint(0, 2**256-1)
A = rn.randint(0, 2**256-1)
M = rn.randint(0, 2**512-1)

print(hex(R))
print(hex(A))
print(hex(M))

In [38]:
def concat(d, bits):
    res = 0
    for i in range(len(d)):
        res += d[i] << (i*bits[i])
    return res

def swe(x, bytes):
    tmp = []
    for i in range(bytes):
        tmp.append((x >> i*8) & 0xFF)

    res = 0
    for i in range(bytes):
        res <<= 8
        res += tmp[i]
    
    return res

In [39]:
print(hex(swe(0x11223344, 4)))

[68, 51, 34, 17]
0x44332211


In [ ]:
print(hex(concat([
    swe(R)
])))

In [4]:
def point_add (x1, y1, z1, t1, x2, y2, z2, t2):
    A = (y1-x1)*(y2-x2) % p
    B = (y1+x1)*(y2+x2) % p
    C = t1*2*d*t2 % p
    D = z1*2*z2 % p
    E = B-A % p
    F = D-C % p
    G = D+C % p
    H = B+A % p
    x3 = E*F % p
    y3 = G*H % p
    t3 = E*H % p
    z3 = F*G % p
    return x3, y3, t3, z3

In [5]:
def point_add_eq(x1, y1, x2, y2):
    x3a = (x1 * y2 + x2 * y1) % p
    x3b = (1 + d * x1 * x2 * y1 * y2) % p
    x3 = (x3a * pow(x3b, p-2, p)) % p

    y3a = (y1 * y2 + x1 * x2) % p
    y3b = (1 - d * x1 * x2 * y1 * y2) % p
    y3 = (y3a * pow(y3b, p-2, p)) % p
    return x3, y3

In [6]:
def shift(x, i):
    return (x >> i) & 1

In [7]:
xG = 0x216936d3cd6e53fec0a4e231fdd6dc5c692cc7609525a7b2c9562d608f25d51a
yG = 0x6666666666666666666666666666666666666666666666666666666666666658
zG = 1
tG = 0x216936d3cd6e53fec0a4e231fdd6dc5c692cc7609525a7b2c9562d608f25d51a * 0x6666666666666666666666666666666666666666666666666666666666666658 % p

In [8]:
def ed_spm(k):
    xQ = 0
    yQ = 1
    zQ = 1
    tQ = 0

    for i in range(255, -1, -1):
        xQ, yQ, zQ, tQ = point_add(xQ, yQ, zQ, tQ, xQ, yQ, zQ, tQ)
        if shift(k, i):
            xQ, yQ, zQ, tQ = point_add(xQ, yQ, zQ, tQ, xG, yG, zG, tG)
    
    return xQ, yQ, zQ, tQ

In [9]:
def ed_spm_eq(k):
    xQ = 0
    yQ = 1

    for i in range(255, -1, -1):
        xQ, yQ = point_add_eq(xQ, yQ, xQ, yQ)
        if shift(k, i):
            xQ, yQ = point_add_eq(xQ, yQ, xG, yG)
    
    return xQ, yQ

In [10]:
x3, y3 = point_add_eq(0, 1, xG, yG)
print(hex(x3))
print(hex(y3))

0x216936d3cd6e53fec0a4e231fdd6dc5c692cc7609525a7b2c9562d608f25d51a
0x6666666666666666666666666666666666666666666666666666666666666658


In [11]:
x3, y3, z3, t3 = point_add(0, 1, 1, 0, xG, yG, zG, tG)
zinv = pow(z3, p-2, p)
print(hex(x3 * zinv % p))
print(hex(xG))
print(hex(y3 * zinv % p))
print(hex(yG))



0x5ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff3
0x216936d3cd6e53fec0a4e231fdd6dc5c692cc7609525a7b2c9562d608f25d51a
0x5272fb06866a6cd6a133e3762c993863943c29930c8453b46731350a14b947df
0x6666666666666666666666666666666666666666666666666666666666666658


In [12]:
k = 0x9d61b19deffd5a60ba844af492ec2cc44449c5697b326919703bac031cae7f60

x3, y3 = ed_spm_eq(k)

print(hex(x3))
print(hex(y3))


0x64c2456540d8b57ab7341be88737310e9122efb77ea911523679fe3edc6ba8d6
0x6f467d17fc26454a1151bbe78da064b2bfcbb16258f0d39ea234592532707a55


In [21]:
hex((p-5)//8)

'0b111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101'

In [14]:
bin(0xd)


'0b1101'

In [15]:
pow(2, (p-1) // 4, p)

19681161376707505956807079304988542015446066515923890162744021073123829784752

In [23]:
b = 0b1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
b - (2**250 - 1)

0

In [26]:
((2**250 - 1) * 2) == 2**251 - 2

True

In [24]:
((p-5)//8) == (2**252 - 3)

True